Implementing linear classifiers

`Group`: PA 4 43

`Date`: 22 February 2023

`Group member`: 
- Albin Ekström
- Jonas Nordin

## Exercise question

The Linear Support Vector Classifier (L-SVC) is based on linear dependence. This means that e.g. a XOR-function can't be memorized by the classifier. It's the same case in the exercise question, the second training set is linearly insepreable, hence it can not be memorized. 

For example, if we try to separate rain and sun using only the city feature, we would have to draw a vertical line between Sydney and Paris, but this line would not separate the rain and sun classes correctly. Similarly, if we try to separate the classes using only the month feature, we would have to draw a horizontal line between July and December, but this line would also be incorrect.

## Linear Classifier

In [17]:
from aml_perceptron import LinearClassifier
import numpy as np

# Pegasos implementation
class LinearSVC(LinearClassifier):
    """
    An implementation of a linear SVC algorithm.
    """

    def __init__(self, n_iter=20, regularization_factor=0.001):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.regularization_factor = regularization_factor

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasos algorithm.
        """
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        XY = list(zip(X, Ye))

        t = 1
        for i in range(self.n_iter):
            for x, y in XY:
                n = 1/(self.regularization_factor * t)

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # Update the weights.
                if y*score < 1:
                    self.w = (1-n*self.regularization_factor)*self.w + (n*y)*x
                else:
                    self.w = (1-n*self.regularization_factor)*self.w
                t += 1


## Linear Regression

In [18]:
# Logisitic regression
from aml_perceptron import LinearClassifier

class LogisticRegression(LinearClassifier):
    """
    An implementation of a logistic regression algorithm.
    """

    def __init__(self, n_iter=20, regularization_factor=0.001):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.regularization_factor = regularization_factor

    def fit(self, X, Y):
            """
            Train a logistic regression.
            """
            # First determine which output class will be associated with positive
            # and negative scores, respectively.
            self.find_classes(Y)

            # Convert all outputs to +1 (for the positive class) or -1 (negative).
            Ye = self.encode_outputs(Y)

            # If necessary, convert the sparse matrix returned by a vectorizer
            # into a normal NumPy matrix.
            if not isinstance(X, np.ndarray):
                X = X.toarray()

            # Initialize the weight vector to all zeros.
            n_features = X.shape[1]
            self.w = np.zeros(n_features)

            XY = list(zip(X, Ye))

            t = 1
            for i in range(self.n_iter):
                loss_values = []
                for x, y in XY:
                    n = 1/(self.regularization_factor * t)

                    # Compute gradient for the loss function
                    loss_grad = -(y / (1 + np.exp(y* (self.w.dot(x)) )))*x

                    # Compute loss function value
                    loss = np.log(1 + np.exp(-y * (self.w.dot(x))))
                    if loss == np.inf: # If loss is too big just set it to a large number
                        loss = 1_000_000
                    loss_values.append(loss)

                    # Compute gradient for the objective function
                    obj_grad = self.regularization_factor * self.w + loss_grad

                    # Update the weight-vector
                    self.w = (1-n*self.regularization_factor)*self.w - n * obj_grad

                    t += 1

                # Print the objective function value
                obj = np.average(loss_values) + self.regularization_factor*((self.w.dot(self.w))**2)
                print(f"Epoch: {i} - Objective function value: {obj}")

## Bonus task 1. Making your code more efficient

Here we're trying to optimize the training algorithms to run faster. We should get the same results with and without the speed improvements. The time measurements and score are presented under the section "Evaluating the classifiers".

### a) Faster linear algebra operations

In [19]:
# Pegasos implementation optimized with faster linear algebra
from scipy.linalg import blas

class LinAlgLinearSVC(LinearClassifier):
    """
    Origninal linear SVC algorithm took 1.69s to run
    This version runs the code in 
    """

    def __init__(self, n_iter=20, regularization_factor=0.001):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.regularization_factor = regularization_factor

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasos algorithm.
        """
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features, dtype = np.float64)

        # Initialize the X and Y as floats
        X = X*1.0
        Ye = Ye*1.0

        # Zip the file before to save process time
        XY = list(zip(X, Ye))

        t = 1
        for i in range(self.n_iter):
            for x, y in XY:
                n = 1/(self.regularization_factor * t)
                constant = (1-n*self.regularization_factor)

                # Compute the output score with the ddot function from BLAS
                score = blas.ddot(x, self.w)
                
                # Update the weights.
                if y*score < 1:
                    blas.dscal(constant, self.w) # w = (1 - n * lambda) * w
                    blas.daxpy(x, self.w, a=(n*y)) # w = w + x * (n*y)
                else:
                    blas.dscal(constant, self.w) # w = (1 - n * lambda) * w
                t += 1


### b) Using sparse vectors

In [20]:
from aml_perceptron import sparse_dense_dot, add_sparse_to_dense
# Pegasos implementation optimized with sparse vectors

class SparseLinearSVC(LinearClassifier):

    def __init__(self, n_iter=20, regularization_factor=0.001):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.regularization_factor = regularization_factor

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasos algorithm.
        """
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))

        t = 1
        for i in range(self.n_iter):
            for x, y in XY:
                n = 1/(self.regularization_factor * t)

                # Compute the output score for this instance.
                score = sparse_dense_dot(x, self.w)

                # Update the weights.
                if y*score < 1:
                    self.w = (1-n*self.regularization_factor)*self.w
                    add_sparse_to_dense(x, self.w, (n*y))
                else:
                    self.w = (1-n*self.regularization_factor)*self.w
                t += 1

### c) Speeding up the scaling operation

In [21]:
# Pegasos implementation optimized with sparse vectors scaling operation

class SparseScalingLinearSVC(LinearClassifier):

    def __init__(self, n_iter=20, regularization_factor=0.001):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.regularization_factor = regularization_factor

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasos algorithm.
        """
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))

        # Intilize the step length t to 2 to avoid zero dividing
        t = 2

        # Initialize scaling factor a to 1
        a = 1
        for i in range(self.n_iter):
            for x, y in XY:
                n = 1/(self.regularization_factor * t)

                # Compute the output score for this instance.
                score = a * sparse_dense_dot(x, self.w)

                # Compute the scaling factor
                a = (1-n*self.regularization_factor)*a

                # Update the weights.
                if y * score < 1:
                    scaling = ((n*y)/a)
                    add_sparse_to_dense(x, self.w, scaling)
                t += 1
        self.w = a * self.w

## Evaluating the Classifiers

In [22]:
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y

if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline_LSVC = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        LinearSVC(n_iter=10, regularization_factor=0.0001)
    )

    pipeline_LINALG = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        LinAlgLinearSVC(n_iter=10, regularization_factor=0.0001)
    )

    pipeline_SPARE = make_pipeline(
        TfidfVectorizer(ngram_range=(1,2)),
        #SelectKBest(k=1000),
        Normalizer(),

        SparseLinearSVC(n_iter=10, regularization_factor=0.0001)
    )

    pipeline_SCALED = make_pipeline(
        TfidfVectorizer(ngram_range=(1,2)),
        #SelectKBest(k=1000),
        Normalizer(),
        
        SparseScalingLinearSVC(n_iter=10, regularization_factor=0.0001)
    )

    pipeline_LR = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),
    
        LogisticRegression(n_iter=10, regularization_factor=0.0001)
    )


    print("Linear SVC classifier: ")

    # Train the classifier.
    t0 = time.time()
    pipeline_LSVC.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline_LSVC.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

    print()
    print("==========================================")
    print()

    print("Linear SVC classifier with fast linear algebra: ")

    # Train the classifier.
    t0 = time.time()
    pipeline_LINALG.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline_LINALG.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

    print()
    print("==========================================")
    print()

    print("Linear SVC classifier with using sparse vectors: ")

    # Train the classifier.
    t0 = time.time()
    pipeline_SPARE.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline_SPARE.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

    print()
    print("==========================================")
    print()

    print("Linear SVC classifier with using sparse vectors and scaling operation: ")

    # Train the classifier.
    t0 = time.time()
    pipeline_SCALED.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline_SCALED.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

    print()
    print("==========================================")
    print()

    print("Logistic Regression: ")

    # Train the classifier.
    t0 = time.time()
    pipeline_LR.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline_LR.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

    print()
    print("==========================================")
    print()



Linear SVC classifier: 
Training time: 1.54 sec.
Accuracy: 0.8351.


Linear SVC classifier with fast linear algebra: 
Training time: 1.22 sec.
Accuracy: 0.8351.


Linear SVC classifier with using sparse vectors: 
Training time: 30.85 sec.
Accuracy: 0.8695.


Linear SVC classifier with using sparse vectors and scaling operation: 
Training time: 4.72 sec.
Accuracy: 0.8703.


Logistic Regression: 


/var/folders/n7/zkztf1z93sb1wpcbr3wrk07c0000gn/T/ipykernel_21365/3845729332.py:49: RuntimeWarning: overflow encountered in exp
  loss = np.log(1 + np.exp(-y * (self.w.dot(x))))


Epoch: 0 - Objective function value: 275.08853119169464
Epoch: 1 - Objective function value: 51.38838675167968
Epoch: 2 - Objective function value: 48.22398057839656
Epoch: 3 - Objective function value: 46.85975799486828
Epoch: 4 - Objective function value: 46.116493261685505
Epoch: 5 - Objective function value: 45.655410353972044
Epoch: 6 - Objective function value: 45.343953770381106
Epoch: 7 - Objective function value: 45.12043130366114
Epoch: 8 - Objective function value: 44.95260141006971
Epoch: 9 - Objective function value: 44.822120370532886
Training time: 2.30 sec.
Accuracy: 0.8292.


